In [1]:
import pandas as pd
import os
import numpy as np
import re
from __future__ import division
import time
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import random
import string

'''
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
random.seed(2016)'''

'\nfrom sklearn.ensemble import RandomForestRegressor, BaggingRegressor\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nrandom.seed(2016)'

# Function for cleaning

In [10]:
def add_zeros_before_point(s):
    if isinstance(s, str):
        #s = re.sub('(^\.[0-9]+)', r' 0\1',s)
        #s = re.sub('( \.[0-9]+)', r' 0\1',s)
        #s = re.findall("[^0-9]*(\.[0-9]+)",s)
        s = re.sub(r"(^| )(\.[0-9]+)",r" 0\2",s)
        return s
    else:
        return 'null'

In [11]:
def calculation(s):
    if isinstance(s, str):
        #num_list = ''
        k = re.findall('[0-9]+ */ *[0-9]+', s)
        for i in k:
            numbers = i.split('/')
            if float(numbers[1]) != 0:
                calculated_num =round(float(numbers[0])/float(numbers[1]),2)
                #num_list = num_list + ' ' + str(calculated_num)
            else:
                calculated_num = float(numbers[0])
            s = re.sub(i, str(calculated_num), s)
        return s
    else:
        return 'null'

In [16]:
def data_cleaning(s):
    if isinstance(s, str):
        s = s.lower()
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("/"," ")
        s = s.replace("\\"," ")
        s = s.replace('&quot;', ' ')
        s = s.replace('&amp', ' ')
        s = s.replace('&#39', ' ')
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = s.replace( "star*", "star")
        s = s.replace( "(e*)", "(e)")
        s = s.replace( "(e)*", "(e)")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sqft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cuft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = add_zeros_before_point(s)
        s = calculation(s)
        return s
    else:
        return 'null'          

In [13]:
def extract_unit(s):
    if isinstance(s, str):
        s = re.findall("[0-9]+\.*[0-9]*[a-z]+\.",s)
        s = ' '.join(s)
        return s
    else:
        return 'null'  
def extract_numbers(s):
    if isinstance(s, str):
        s = re.findall("[0-9]+\.*[0-9]* ",s)
        s = ' '.join(s)
        return s
    else:
        return 'null'  

In [14]:
stemmer = SnowballStemmer('english')
def text_processing(s):
    if isinstance(s, str):
        #s = s.replace('*','x')
        s = re.sub('[0-9]', ' ', s)
        s = s.lower()
        #s = s.replace('in.', 'inch')
        s = s.replace('-', ' ')
        s = s.translate(None, string.punctuation)
        tokens = nltk.word_tokenize(s)
        return ' '.join([stemmer.stem(word.decode('utf-8',errors='ignore')) for word in tokens])
    else:
         return 'null'  

# load Data

In [7]:
product = pd.read_csv('./Dataset/product.csv')
search_term = pd.read_csv('./Dataset/search.csv')

In [9]:
search_term.info(),product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240760 entries, 0 to 240759
Data columns (total 4 columns):
id                 240760 non-null int64
product_uid        240760 non-null int64
search_term_new    240760 non-null object
relevance          74067 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 9.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 124428 entries, 0 to 124427
Data columns (total 5 columns):
prod_id                124428 non-null int64
product_title          124428 non-null object
product_description    124428 non-null object
brand                  124428 non-null object
attribute              86263 non-null object
dtypes: int64(1), object(4)
memory usage: 5.7+ MB


(None, None)

# Process search term

In [20]:
search = search_term[['id','product_uid','relevance']].copy()
#search_term['search_term_new'] = search_term['search_term_new'].apply(lambda s: add_zeros_before_point(s))
#search_term['search_term_new'] = search_term['search_term_new'].apply(lambda s: calculation(s))
search['search_term'] = search_term['search_term_new'].apply(lambda s: data_cleaning(s))
search['search_term_unit'] = search_term['search_term_new'].apply(lambda s: extract_unit(s))
search['search_term_number'] = search_term['search_term_new'].apply(lambda s: extract_numbers(s))

In [21]:
search.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240760 entries, 0 to 240759
Data columns (total 6 columns):
id                    240760 non-null int64
product_uid           240760 non-null int64
relevance             74067 non-null float64
search_term           240760 non-null object
search_term_unit      240760 non-null object
search_term_number    240760 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 12.9+ MB


In [22]:
search['search_term_stemmed'] = search.search_term.apply(lambda s: text_processing(s))

In [23]:
search.drop('search_term',axis=1).to_csv('./Dataset/search_cleaned_stemmed.csv', index=False)

# process product

In [30]:
product_new = product[['prod_id']].copy()

In [32]:
#product['product_title'] = product['product_title'].apply(lambda s: add_zeros_before_point(s))
#product['product_title'] = product['product_title'].apply(lambda s: calculation(s)) 
product_new['product_title'] = product['product_title'].apply(lambda s: data_cleaning(s))
product_new['product_title_unit'] = product['product_title'].apply(lambda s: extract_unit(s)) 
product_new['product_title_number'] = product['product_title'].apply(lambda s: extract_numbers(s)) 
product_new['product_title_stemmed'] = product['product_title'].apply(lambda s: text_processing(s)) 

In [34]:
#product['product_description'] = product['product_description'].apply(lambda s: add_zeros_before_point(s))
#product['product_description'] = product['product_description'].apply(lambda s: calculation(s)) 
product_new['product_description'] = product['product_description'].apply(lambda s: data_cleaning(s))
product_new['product_description_unit'] = product['product_description'].apply(lambda s: extract_unit(s)) 
product_new['product_description_number'] = product['product_description'].apply(lambda s: extract_numbers(s)) 
product_new['product_description_stemmed'] = product['product_description'].apply(lambda s: text_processing(s)) 

In [244]:
#not use
'''#product['attribute'] = product['attribute'].apply(lambda s: add_zeros_before_point(s))
#product['attribute'] = product['attribute'].apply(lambda s: calculation(s)) 
product_new['attribute'] = product['attribute'].apply(lambda s: data_cleaning(s))
product_new['attribute_unit'] = product['attribute'].apply(lambda s: extract_unit(s)) 
product_new['attribute_number'] = product['attribute'].apply(lambda s: extract_numbers(s)) 
product_new['attribute_stemmed'] = product['attribute'].apply(lambda s: text_processing(s)) '''

In [36]:
product_new['brand_stemmed'] = product['brand'].apply(lambda s: text_processing(s))

In [38]:
product_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124428 entries, 0 to 124427
Data columns (total 10 columns):
prod_id                        124428 non-null int64
product_title                  124428 non-null object
product_title_unit             124428 non-null object
product_title_number           124428 non-null object
product_title_stemmed          124428 non-null object
product_description            124428 non-null object
product_description_unit       124428 non-null object
product_description_number     124428 non-null object
product_description_stemmed    124428 non-null object
brand_stemmed                  124428 non-null object
dtypes: int64(1), object(9)
memory usage: 10.4+ MB


In [43]:
product_new.drop(['product_title','product_description'],axis=1).to_csv('Dataset/product_stemmed.csv',index=False,encoding = 'utf-8')

# other things

In [257]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240760 entries, 0 to 240759
Data columns (total 7 columns):
﻿id                    240760 non-null int64
product_uid            240760 non-null int64
search_term_new        240760 non-null object
relevance              74067 non-null float64
search_term_unit       240760 non-null object
search_term_number     240760 non-null object
search_term_stemmed    240760 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 14.7+ MB


In [100]:
features = pd.DataFrame()
data_merged = pd.DataFrame()

In [256]:
product = product.rename(columns={'pro_id': 'product_id'}, inplace=True)

In [254]:
# mapping search term & product
data = search_term.copy()
data= pd.merge(search_term, product, left_on='product_uid', right_on='product_id', how='left')

KeyError: 'product_id'

In [111]:
def str_common_word(str1, str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())

In [130]:
def common_word_percent(str1, str2):
    count = 0
    if len(str1) != 0:
        for word in str1.lower().split():
            if str2.lower().find(word) >= 0:
                count += 1
            else:
                count = count
            #count_sum = count_sum + count
        percent_included = count/(len(str1.lower().split()))
        return percent_included
    else:
        return 0

In [119]:
data['word_in_title'] = data[['product_title_stemmed','search_term_stemmed']].apply(lambda s:str_common_word(s['search_term_stemmed'],
                                                                                                            s['product_title_stemmed']),axis=1)

In [121]:
data['word_in_desc'] = data[['product_description_stemmed','search_term_stemmed']].apply(lambda s:str_common_word(s['search_term_stemmed'],
                                                                                                            s['product_description_stemmed']),axis=1)

In [124]:
data['word_in_attr'] = data[['attribute_stemmed','search_term_stemmed']].apply(lambda s:str_common_word(s['search_term_stemmed'],
                                                                                                            s['attribute_stemmed']),axis=1)

In [125]:
data['word_in_brand'] = data[['brand_stemmed','search_term_stemmed']].apply(lambda s:str_common_word(s['search_term_stemmed'],
                                                                                                            s['brand_stemmed']),axis=1)

In [131]:
data['percent_in_title'] = data[['product_title_stemmed','search_term_stemmed']].apply(lambda s:common_word_percent(s['search_term_stemmed'],
                                                                                                            s['product_title_stemmed']),axis=1)
data['percent_in_desc'] = data[['product_description_stemmed','search_term_stemmed']].apply(lambda s:common_word_percent(s['search_term_stemmed'],
                                                                                                            s['product_description_stemmed']),axis=1)
data['percent_in_attr'] = data[['attribute_stemmed','search_term_stemmed']].apply(lambda s:common_word_percent(s['search_term_stemmed'],
                                                                                                            s['attribute_stemmed']),axis=1)
data['percent_in_brand'] = data[['brand_stemmed','search_term_stemmed']].apply(lambda s:common_word_percent(s['search_term_stemmed'],
                                                                                                            s['brand_stemmed']),axis=1)

In [132]:
data['brand_in_search'] =  data[['brand_stemmed','search_term_stemmed']].apply(lambda s:common_word_percent(s['brand_stemmed'],
                                                                                                           s['search_term_stemmed']),axis=1)

In [134]:
# match unit
data['unit_all'] = data.search_term_unit + '\t' + data.product_title_unit + '\t' + data.product_description_unit + '\t' + data.attribute_unit

In [135]:
def unit_title(s):
    title = s.split('\t')[1]
    search = s.split('\t')[0]
    if search == '':
        return 1
    elif search in title:
        return 2
    else:
        return 0
def unit_desc(s):
    title = s.split('\t')[2]
    search = s.split('\t')[0]
    if search == '':
        return 1
    elif search in title:
        return 2
    else:
        return 0    
def unit_attr(s):
    title = s.split('\t')[3]
    search = s.split('\t')[0]
    if search == '':
        return 1
    elif search in title:
        return 2
    else:
        return 0  

In [136]:
data['unit_title'] = data.unit_all.apply(lambda s:unit_title(s))
data['unit_desc'] = data.unit_all.apply(lambda s:unit_desc(s))
data['unit_attr'] = data.unit_all.apply(lambda s:unit_attr(s))

In [138]:
# match numbers
data['num_all'] = data.search_term_number + '\t' + data.product_title_number + '\t'+data.product_description_number+'\t'+data.attribute_number

In [155]:
def num_title(s):
    title = s.split('\t')[1]
    search = s.split('\t')[0]
    if search == '':
        return 1
    elif len(search.split()) == 1:
        num = search.split()[0] in title.split()
        if num == True:
            return 2
        else:
            return 0
    elif search in title:
        return 3
    else:
        return 0
def num_desc(s):
    title = s.split('\t')[2]
    search = s.split('\t')[0]
    if search == '':
        return 1
    elif len(search.split()) == 1:
        num = search.split()[0] in title.split()
        if num == True:
            return 2
        else:
            return 0
    elif search in title:
        return 3
    else:
        return 0
def num_attr(s):
    title = s.split('\t')[3]
    search = s.split('\t')[0]
    if search == '':
        return 1
    elif len(search.split()) == 1:
        num = search.split()[0] in title.split()
        if num == True:
            return 2
        else:
            return 0
    elif search in title:
        return 3
    else:
        return 0

In [156]:
data['num_title'] = data.num_all.apply(lambda s:num_title(s))
data['num_desc'] = data.num_all.apply(lambda s:num_desc(s))
data['num_attr'] = data.num_all.apply(lambda s:num_attr(s))

In [159]:
# find common word 
def common_word(str1, str2):
    content =''
    for word in str1.split():
        if word in str2.split():
            content = content + ' ' + word
    return content

In [164]:
# common word in title
data['title_search'] = data['search_term_stemmed'] + '\t' + data['product_title_stemmed']
data['word_in_title_tfidf'] = data['title_search'].map(lambda x:common_word(x.split('\t')[0],x.split('\t')[1]))
search_idf_list = list(data['word_in_title_tfidf']) 
title_stemmed_list = list(data['product_title_stemmed'])

In [225]:
# common word in desc
data['desc_search'] = data['search_term_stemmed'] + '\t' + data['product_description_stemmed']
data['word_in_desc_tfidf'] = data['desc_search'].map(lambda x:common_word(x.split('\t')[0],x.split('\t')[1]))
search_idf_list_desc = list(data['word_in_desc_tfidf']) 
desc_stemmed_list = list(data['product_description_stemmed'])
# common word in attr
data['attr_search'] = data['search_term_stemmed'] + '\t' + data['attribute_stemmed']
data['word_in_attr_tfidf'] = data['attr_search'].map(lambda x:common_word(x.split('\t')[0],x.split('\t')[1]))
search_idf_list_attr = list(data['word_in_attr_tfidf']) 
attr_stemmed_list = list(data['attribute_stemmed'])

In [224]:
# tfidf for title
tfidf = TfidfVectorizer()
tfs = tfidf.fit_transform(product['product_title_stemmed'])
feature_names = tfidf.get_feature_names()
tfs.shape
# tfidf for desc
tfidf_desc = TfidfVectorizer()
tfs_desc = tfidf_desc.fit_transform(product['product_description_stemmed'])
feature_names_desc = tfidf_desc.get_feature_names()
tfs_desc.shape
# tfidf for attr
tfidf_attr = TfidfVectorizer()
tfs_attr = tfidf_attr.fit_transform(product['attribute_stemmed'])
feature_names_attr = tfidf_attr.get_feature_names()
tfs_attr.shape

(124428, 21721)

In [165]:
# title
weight_list =[]
for i in range(len(data)):
    if search_idf_list[i].split() != []:
        weight = 0
        sum_weight = 0
        for word in search_idf_list[i].split():
            if len(word) > 1:
                word_index = feature_names.index(word)
                response = tfidf.transform([title_stemmed_list[i]])
                weight = response[0,word_index]
                sum_weight += weight
    else: sum_weight = 0
    weight_list += [sum_weight]

In [226]:
# desc
weight_list_desc =[]
for i in range(len(search_idf_list_desc)):
    if search_idf_list_desc[i].split() != []:
        weight = 0
        sum_weight = 0
        for word in search_idf_list_desc[i].split():
            if len(word) > 1:
                word_index = feature_names_desc.index(word)
                response = tfidf_desc.transform([desc_stemmed_list[i]])
                weight = response[0,word_index]
                sum_weight += weight
    else: sum_weight = 0
    weight_list_desc += [sum_weight]

In [227]:
# attr
weight_list_attr =[]
for i in range(len(search_idf_list_attr)):
    if search_idf_list_attr[i].split() != []:
        weight = 0
        sum_weight = 0
        for word in search_idf_list_attr[i].split():
            if len(word) > 1:
                word_index = feature_names_attr.index(word)
                response = tfidf_attr.transform([attr_stemmed_list[i]])
                weight = response[0,word_index]
                sum_weight += weight
    else: sum_weight = 0
    weight_list_attr += [sum_weight]

In [228]:
data['tfidf_desc'] = weight_list_desc
data['tfidf_attr'] = weight_list_attr
features['tfidf_desc'] = weight_list_desc
features['tfidf_attr'] = weight_list_attr

In [229]:
features.to_csv('./features_with_tfidf_attr.csv', index=False)

In [167]:
data['tfidf_title'] = weight_list

In [170]:
features = data[['word_in_title','word_in_desc','word_in_attr','word_in_brand','percent_in_title',
                'percent_in_desc','percent_in_attr','percent_in_brand','brand_in_search',
                'unit_title','unit_desc','unit_attr','num_title','num_desc','num_attr',
                'tfidf_title']].copy()

In [5]:
#train & test
num_train = 74067
train_x = features.iloc[:num_train]
test_x= features.iloc[num_train:]
train_y = data['relevance'].iloc[:num_train]

In [6]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(train_x, train_y)
y_pred = clf.predict(test_x)

In [7]:
prediction = pd.DataFrame(y_pred)

In [8]:
prediction.to_csv('./submission_14_tfidf_without_attr.csv', index=False)

In [179]:
features.to_csv('./features.csv')

In [200]:
# tfidf for search terms
#tfidf_search = TfidfVectorizer()
#tfs_search = tfidf_search.fit_transform(data['search_term_stemmed'])
#feature_names_search = tfidf_search.get_feature_names()
#tfs_search.shape

(240760, 6910)

In [202]:
'''
weight_list =[]
for i in range(len(data)):
    if search_idf_list[i].split() != []:
        weight = 0
        sum_weight = 0
        for word in search_idf_list[i].split():
            if len(word) > 1:
                word_index = feature_names.index(word)
                response = tfidf.transform([title_stemmed_list[i]])
                weight = response[0,word_index]
                word_index_search = feature_names_search.index(word)
                response_search = tfidf_search.transform([search_idf_list[i]])
                weight_search = response_search[0,word_index_search]
                weight_new = weight * weight_search
                sum_weight += weight_new
    else: sum_weight = 0
    weight_list += [sum_weight]
'''

In [203]:
#data['tfidf_title_times_search'] = weight_list

In [204]:
#features['tfidf_title_times_search'] = weight_list

In [219]:
#testing gradient boosting regression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor


In [220]:
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1,
                                random_state=0, loss='ls').fit(train_x, train_y)
prediction = est.predict(test_x)

In [221]:
prediction = pd.DataFrame(prediction)

In [222]:
prediction.to_csv('./submission_13.csv', index=False)